In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ray on Vertex cluster management


<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex/ray_cluster_management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex/ray_cluster_management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/ray_on_vertex/ray_cluster_management.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This tutorial demonstrates how to use Ray on Vertex SDK for cluster management.

Learn more about [Ray on Vertex overview](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview).

### Objective

In this tutorial, you learn how to creating cluster, listing clusters, getting a cluster, updating (manually scaling) cluster, and deleting cluster.

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Training`
- [`Ray on Vertex`](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview)


The steps performed include:

- Create a cluster.
- List existing clusters.
- Get a cluster.
- Manually scale up the cluster, then scale down the cluster.
- Delete the cluster.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Ray on Vertex AI pricing](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview#pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[ray]

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

- *{Note to notebook author: For any user-provided strings that need to be unique (like bucket names or model ID's), append "-unique" to the end so proper testing can occur}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
from google.cloud import aiplatform
import vertex_ray

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

[Set up a VPC peering network](https://cloud.google.com/vertex-ai/docs/general/vpc-peering) and private services connection to access Vertex AI.

In [ ]:
# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

VPC_NETWORK = "[your-network-name]"
VPC_NETWORK_FULL = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, VPC_NETWORK)
VPC_NETWORK_FULL

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Create a cluster

Note that within the same VPC network, numbers of clusters and nodes you can create are restricted by IP ranges.

In [ ]:
head_node_type = vertex_ray.Resources(
    machine_type="n1-standard-4",
    node_count=1,
)

worker_node_types = [vertex_ray.Resources(
    machine_type="n1-standard-8",
    node_count=2,  # Can be > 1
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1,
)]

cluster_resource_name = vertex_ray.create_ray_cluster(
    head_node_type=head_node_type,
    network=VPC_NETWORK_FULL,
    worker_node_types=worker_node_types,
)

## List existing clusters

In [ ]:
clusters = vertex_ray.list_ray_clusters()
clusters

## Update an existing cluster (manually scaling)


Note that the maximum number of worker nodes you can scale up depens on these [formulas](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/set-up) and is restricted by IP ranges within the same VPC network.

Get the cluster you want to scale.

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

Scale down workers from 2 nodes to 1 node.

In [ ]:
new_worker_node_types = []
for worker_node_type in cluster.worker_node_types:
    worker_node_type.node_count = 1
    new_worker_node_types.append(worker_node_type)

cluster_resource_name = vertex_ray.update_ray_cluster(
    cluster_resource_name=cluster_resource_name,
    worker_node_types=new_worker_node_types,
)

Verify if the cluster is successfully scaled down.

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

Scale up to 3 worker nodes.

In [ ]:
new_worker_node_types = []
for worker_node_type in cluster.worker_node_types:
    worker_node_type.node_count = 3
    new_worker_node_types.append(worker_node_type)

cluster_resource_name = vertex_ray.update_ray_cluster(
    cluster_resource_name=cluster_resource_name,
    worker_node_types=new_worker_node_types,
)

Verify if the cluster is successfully scaled up.

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the cluster you created in this tutorial.

In [ ]:
import os

# Delete a cluster
vertex_ray.delete_ray_cluster(clusters[0].cluster_resource_name)

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI